# Search Tester Analysis
*Follow these steps to compare legacy results to the new api results.*

### Setup the environment

In [1]:
%run ./local/setup.ipynb

Unable to configure logging, attempted conf:/home/notebook/.local/lib/python3.10/site-packages/ppr_api/logging.conf


### Find search batches (choose the data set)
*find the ones you want to analyze*

In [3]:
search_type = None
after_date = None
before_date = None
### OPTIONAL: refine your batch search with any of the following
# search_type = SearchRequest.SearchTypes.XXXX
# before_date = _datetime_in_utc_
after_date = "2022-01-17 01:20:21.936128"
batches = TestSearchBatch.find_search_batches(search_type, after_date, before_date)
for batch in batches:
    print(batch.id, batch.test_date, batch.search_type, batch.sim_val_business, batch.sim_val_first_name, batch.sim_val_last_name)


700 2022-01-17 19:16:43.559239 BS 0.6 0.4 0.29
703 2022-01-17 19:30:01.579660 IS 0.6 0.4 0.29
705 2022-01-17 21:47:59.334789 SS 0.6 0.4 0.29


In [7]:
# add the ids of the batches you want to analyze
batch_ids = []
for batch in batches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. batch.id > _ , batch.sim_val_business > _ ...
        batch_ids.append(batch.id)

### Get the batch json

In [8]:
batches_to_analyze = []
for batch_id in batch_ids:
    batches_to_analyze.append(TestSearchBatch.find_by_id(batch_id))

# Analysis

### Exact Matches

#### Pass rate

In [9]:
searches_total = 0
searches_passed = 0
failed_searches = []
all_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        all_searches.append(search)
        if len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  0.9095477386934674
number of failed searches:  144


#### Failed Searches

*select failed searches to analyze*

In [10]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesExact']['resultsLegacy']), 'missed: ', len(json['matchesExact']['missedMatches']))

###########################################################
criteria:  RYAN COMPANY LIMITED                                                  
total expected:  44 missed:  15
###########################################################
criteria:  0827532 B.C. LTD.                                                     
total expected:  1 missed:  1
###########################################################
criteria:  675613 B.C. LTD.                                                      
total expected:  1 missed:  1
###########################################################
criteria:  0723787 BC LTD.                                                       
total expected:  1 missed:  1
###########################################################
criteria:  R.L.C. ENTERPRIZE LTD.                                                
total expected:  17 missed:  2
###########################################################
criteria:  0957080 B.C. LTD.                                                 

###########################################################
criteria:  BLUE                     DARRIN         CAMERON                       
total expected:  2 missed:  2
###########################################################
criteria:  BROWN                    RYAN                                         
total expected:  25 missed:  1
###########################################################
criteria:  MITCHELL                 KRAY                                         
total expected:  1 missed:  1
###########################################################
criteria:  WANG                     TIAN           WEI                           
total expected:  3 missed:  3
###########################################################
criteria:  GUO                      YU             SHUN                          
total expected:  8 missed:  7
###########################################################
criteria:  LEE                      CINDY          L                           

###########################################################
criteria:  ALEXANDER                RHYANNON                                     
total expected:  3 missed:  2
###########################################################
criteria:  ST JOHN                  JOEL                                         
total expected:  2 missed:  2
###########################################################
criteria:  BROWN                    TED                                          
total expected:  3 missed:  1
###########################################################
criteria:  GILL                     JAGDEEP        SINGH                         
total expected:  43 missed:  5
###########################################################
criteria:  GILL                     AMARJIT        SINGH                         
total expected:  13 missed:  2
###########################################################
criteria:  GILL                     GURPREET       KAUR                       

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [11]:
exact_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > _
        exact_searches_analysis.append(search)

*missed matches*

In [14]:
import re
### manually iterate through exact_searches_analysis one by one to see the missed matches
# search = exact_searches_analysis[1]
# for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])
### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

##################################################################################
criteria:  RYAN COMPANY LIMITED                                                  
Missed Matches
-------------------------------------------------------------------------------
result: RYAN  COMPANY LTD                                              
reg num: 982004L
index:  0
-------------------------------------------------------------------------------
result: RYAN COMPANY LTD                                               
reg num: 521297L
index:  27
-------------------------------------------------------------------------------
result: RYAN COMPANY LTD                                               
reg num: 826454L
index:  28
-------------------------------------------------------------------------------
result: RYAN COMPANY LTD.                                              
reg num: 402510J
index:  31
-------------------------------------------------------------------------------
result: RYAN COMPANY 

-------------------------------------------------------------------------------
result: 0911488 BC LTD                                                 
reg num: 512098G
index:  0
##################################################################################
criteria:  0991557 B.C. LTD.                                                     
Missed Matches
-------------------------------------------------------------------------------
result: 0991557 B.C. LTD.                                              
reg num: 782412H
index:  0
##################################################################################
criteria:  THE SHORE GP INC.                                                     
Missed Matches
-------------------------------------------------------------------------------
result: THE SHORE GP INC                                               
reg num: 859009K
index:  0
-------------------------------------------------------------------------------
result: THE SHORE GP IN

-------------------------------------------------------------------------------
result: LEWIS                    J                              75JUL28
reg num: 404887M
index:  0
-------------------------------------------------------------------------------
result: LEWIS                    J                              78DEC10
reg num: 798253J
index:  1
-------------------------------------------------------------------------------
result: LEWIS                    J              FRANKLIN        50APR19
reg num: 303791M
index:  2
##################################################################################
criteria:  GILL                     HARPALJIT                                    
Missed Matches
-------------------------------------------------------------------------------
result: GILL                     H                              97APR10
reg num: 270218L
index:  0
-------------------------------------------------------------------------------
result: GILL            

-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         40DEC09
reg num: 388743E
index:  0
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         50JUN28
reg num: 877208K
index:  1
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         50OCT02
reg num: 158456L
index:  2
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         51FEB10
reg num: 872963L
index:  3
-------------------------------------------------------------------------------
result: BROWN                    ROBERT                         51FEB10
reg num: 376839M
index:  4
-------------------------------------------------------------------------------
result: BROWN            

-------------------------------------------------------------------------------
result: CENTER                   CHRISTOPHER    KENT            75DEC06
reg num: 769800I
index:  0
-------------------------------------------------------------------------------
result: CENTER                   CHRISTOPHER    KENT            75DEC06
reg num: 553828M
index:  1
##################################################################################
criteria:  BLUE                     DEBORAH        ANN                           
Missed Matches
-------------------------------------------------------------------------------
result: BLUE                     DEBORAH                               
reg num: 857170G
index:  0
-------------------------------------------------------------------------------
result: BLUE                     DEBORAH                        62JAN17
reg num: 322244I
index:  1
##################################################################################
criteria:  BLUE      

...

-------------------------------------------------------------------------------
result: KHAN                     MOHAMMED                              
reg num: 943545L
index:  0
-------------------------------------------------------------------------------
result: KHAN                     MOHAMMED                              
reg num: 943562L
index:  1
-------------------------------------------------------------------------------
result: KHAN                     MOHAMMED                              
reg num: 943581L
index:  2
-------------------------------------------------------------------------------
result: KHAN                     MOHAMMED                       62MAY19
reg num: 680810L
index:  3
-------------------------------------------------------------------------------
result: KHAN                     MOHAMMED                       64MAR06
reg num: 688779I
index:  4
-------------------------------------------------------------------------------
result: KHAN             

-------------------------------------------------------------------------------
result: GILL                     JASKARAN                              
reg num: 338146E
index:  0
##################################################################################
criteria:  MCLEAN                   RODNEY                                       
Missed Matches
-------------------------------------------------------------------------------
result: MCLEAN                   R-DOUGLAS      DOUGLAS         59JUN27
reg num: 110413L
index:  0
##################################################################################
criteria:  WRUTH                    DARON          ROBERT                        
Missed Matches
-------------------------------------------------------------------------------
result: WRUTH                    DARON                          61MAR14
reg num: 276235L
index:  0
-------------------------------------------------------------------------------
result: WRUTH          

-------------------------------------------------------------------------------
result: ROBERTSON                JASON                          69JUN20
reg num: 516655M
index:  0
-------------------------------------------------------------------------------
result: ROBERTSON                JASON                          73JAN11
reg num: 973699K
index:  1
-------------------------------------------------------------------------------
result: ROBERTSON                JASON          C               75APR16
reg num: 805460L
index:  2
##################################################################################
criteria:  BROWN                    TED                                          
Missed Matches
-------------------------------------------------------------------------------
result: BROWN                    T              A               66NOV19
reg num: 126612M
index:  0
##################################################################################
criteria:  BROWN     

-------------------------------------------------------------------------------
result: SANGHERA                 MANDEEP        SINGH           85JAN11
reg num: 710919K
index:  1
-------------------------------------------------------------------------------
result: SANGHERA                 MANDEEP        SINGH           85JAN11
reg num: 065822L
index:  2
-------------------------------------------------------------------------------
result: SANGHERA                 MANDEEP        SINGH           86SEP17
reg num: 968148I
index:  4
##################################################################################
criteria:  GODIN                    RICHARD        DANIEL                        
Missed Matches
-------------------------------------------------------------------------------
result: GODIN                    RICHARD                        61OCT09
reg num: 793602L
index:  0
-------------------------------------------------------------------------------
result: GODIN           

-------------------------------------------------------------------------------
result: MV 1XKDP4TX3ER966668         2014 KENWORTH T800                
reg num: 563984M
index:  0
##################################################################################
criteria:  20001-1252 E6I6-2909                                                  
Missed Matches
-------------------------------------------------------------------------------
result: MV 20001-1252 E6I6-2909      1996 LINK-BELT 2800               
reg num: 563984M
index:  0
##################################################################################
criteria:  1FMCU9J90JUA01027                                                     
Missed Matches
-------------------------------------------------------------------------------
result: MV TG88B-TT-IH-GI-0038       1998 TRANS GESCO TG88B-TT         
reg num: 563984M
index:  0
##################################################################################
criteria:  1FUJHHDR2

*results diff*

In [16]:
import re
### manually iterate through exact_searches_analysis one by one
# search = exact_searches_analysis[1]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('-------------------------------------------------------------')
    print('  legacy                           api')
    print('-------------------------------------------------------------')
    legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
    api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
    length = max(len(legacy_results), len(api_results))
    for i in range(length):
        if i < len(legacy_results) and i < len(api_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
        elif i < len(legacy_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
        elif i < len(api_results):
            print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')


##################################################################################
criteria:  RYAN COMPANY LIMITED                                                  
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 982004L RYAN COMPANY LTD  | 597683L RYAN COMPANY LIMITED
1: 673482F RYAN COMPANY LIMITED  | 127138L RYAN COMPANY LIMITED
2: 742785J RYAN COMPANY LIMITED  | 575024L RYAN COMPANY LIMITED
3: 381798K RYAN COMPANY LIMITED  | 611621M RYAN COMPANY LIMITED
4: 381803K RYAN COMPANY LIMITED  | 673482F RYAN COMPANY LIMITED
5: 619029K RYAN COMPANY LIMITED  | 584620M RYAN COMPANY LIMITED
6: 817169K RYAN COMPANY LIMITED  | 742785J RYAN COMPANY LIMITED
7: 064227L RYAN COMPANY LIMITED  | 381798K RYAN COMPANY LIMITED
8: 064238L RYAN COMPANY LIMITED  | 381803K RYAN COMPANY LIMITED
9: 108249L RYAN COMPANY LIMITED  | 245281M RYAN COMPANY LIMITED
10: 127138L RYAN COMPANY LIMITED  | 

0: 859009K THE SHORE GP INC  |
1: 944175L THE SHORE GP INC.  |
2: 944175L THE SHORE GP INC.  |
##################################################################################
criteria:  THE OWNERS STRATA PLAN EPS 6752                                       
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 522145M THE OWNERS STRATA PLAN EPS 6752  |
1: 522150M THE OWNERS STRATA PLAN EPS 6752  |
##################################################################################
criteria:  NOR LAND LIMITED                                                      
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 5177917 NOR LAND LIMITED  | 416367M NOR LAND LIMITED
1: 6828427 NOR LAND LIMITED  | 5177917 NOR LAND LIMITED
2: 027755I NOR LAND LIMITED  | 6828427 NOR LA

0: 453601I 1025532 B.C. LTD.  | 453601I 1025532 B.C. LTD.
1: 561013I 1025532 B.C. LTD.  | 561013I 1025532 B.C. LTD.
2: 089452M 1025532 B.C. LTD.  | 089452M 1025532 B.C. LTD.
3: 453601I 1025532 BC LTD  |
4: 561013I 1025532 BC LTD  |
5: 609300D 1025532 BC LTD  |
6: 722931H 1025532 BC LTD  |
7: 208373H 1025532 BC LTD  |
8: 722973H 1025532 BC LTD  |
9: 859274K 1025532 BC LTD  |
##################################################################################
criteria:  0733680 B.C. LTD                                                      
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 669189L 0733680 B.C. LTD.  |
1: 669339L 0733680 B.C. LTD.  |
2: 727796L 0733680 B.C. LTD.  |
##################################################################################
criteria:  1163750 B.C. LTD.                                                     
----------------------------------

0: 491367L CHEN CHIA 44JUL24 |
1: 290427L CHEN CHIA 66OCT19 |
##################################################################################
criteria:  CHEN                     CHIA           SHU                           
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 491367L CHEN CHIA 44JUL24 |
1: 290427L CHEN CHIA 66OCT19 |
##################################################################################
criteria:  SMITH                    DAROL                                        
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 132522M SMITH D ALLAN 56MAR05 | 587716G SMITH DAROL KIRK
1: 079753M SMITH D BRIAN 92OCT09 | 823665K SMITH DAROL KIRK
2: 860499K SMITH D W 47SEP24 | 587724G SMITH DAROL KIRK
3: 587716G SMITH DAROL KIRK 69AUG23 |
4: 587

0: 692626L WANG TIAN 87JUL09 |
1: 063776M WANG TIAN 89FEB05 |
2: 301408M WANG TIAN 92MAY28 |
##################################################################################
criteria:  GUO                      YU             SHUN                          
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 913408L GUO YU 67JUN06 | 443627L GUO YU SHUN
1: 544492L GUO YU 84JAN05 |
2: 780275L GUO YU 84MAR24 |
3: 736220L GUO YU 89AUG31 |
4: 437524M GUO YU 90APR08 |
5: 570502M GUO YU 93MAY18 |
6: 837193L GUO YU 99OCT18 |
7: 443627L GUO YU SHUN 91FEB16 |
##################################################################################
criteria:  LEE                      CINDY          L                             
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0:

...

0: 447531L MADAHAR JASWINDER SINGH  |
##################################################################################
criteria:  BROWN                    JOHN           RUFUS                         
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 481822M BROWN JOHN 65APR24 |
1: 228110L BROWN JOHN 65DEC07 |
2: 374172J BROWN JOHN 68MAY13 |
3: 473587L BROWN JOHN 68MAY13 |
4: 066852M BROWN JOHN 68MAY13 |
5: 440878K BROWN JOHN 82JUL31 |
##################################################################################
criteria:  CHAPMAN                  DAVID          MARK                          
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 245538K CHAPMAN DAVID 67MAY13 |
###############################################################################

0: 058792M ST JOHN JOEL 59JAN09 |
1: 924169K ST JOHN JOEL HOWARD 59JAN09 |
##################################################################################
criteria:  BROWN                    TED                                          
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 126612M BROWN T A 66NOV19 | 647482L BROWN TED 
1: 647482L BROWN TED  | 647501L BROWN TED 
2: 647501L BROWN TED  | 920110M BROWN TED D
##################################################################################
criteria:  GILL                     JAGDEEP        SINGH                         
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 032996M GILL JAGDEEP 70APR08 | 718754K GILL JAGDEEP SINGH
1: 450923H GILL JAGDEEP 76APR27 | 828734M GILL JAGDEEP SINGH
2: 055537K

0: 016188M BROWN STEVEN  | 076888M BROWN STEVEN TED
1: 331373L BROWN STEVEN 65MAY09 | 647501L BROWN STEVEN TED
2: 647501L BROWN STEVEN TED  |
3: 076888M BROWN STEVEN TED 74MAY16 |
##################################################################################
criteria:  BROWN                    STEVEN                                       
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 894054K BROWN S MAHLI 71OCT01 | 331373L BROWN STEVEN 
1: 016188M BROWN STEVEN  | 949475M BROWN STEVEN 
2: 331373L BROWN STEVEN 65MAY09 | 949449M BROWN STEVEN RILEY
3: 489192K BROWN STEVEN A 60JUL08 | 789376J BROWN STEVEN ANDREW
4: 789376J BROWN STEVEN ANDREW 72FEB16 | 489192K BROWN STEVEN A
5: 730334I BROWN STEVEN ANTHONY 81JUN10 | 016188M BROWN STEVEN 
6: 446279L BROWN STEVEN BLACKWELL 93SEP24 | 358142I BROWN STEVEN WILFRED
7: 497955L BROWN STEVEN BLACKWELL 93SEP24 | 061207L BROWN ST

0: 016188M BROWN STEVEN  |
1: 331373L BROWN STEVEN 65MAY09 |
##################################################################################
criteria:  BROWN                    STEVEN         THEDORE                       
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 016188M BROWN STEVEN  |
1: 331373L BROWN STEVEN 65MAY09 |
##################################################################################
criteria:  TODD                     RICHARD        ALLEN                         
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 925463J TODD RICHARD 45SEP11 |
1: 859012K TODD RICHARD 54SEP09 |
2: 234056M TODD RICHARD 54SEP09 |
3: 257894L TODD RICHARD A 54SEP09 |
##################################################################################
c

0: 563984M MV K6J2-6202 2002 LINK-BELT 330LX  |
##################################################################################
criteria:  DDM-0018                                                              
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 083945M TR DDM-0018 DISCOVERY DIAMOND DRILL  |
1: 083999M TR DDM-0018 DISCOVERY DIAMOND DRILL  |
##################################################################################
criteria:  5KKPALBG9DPFA6993                                                     
-------------------------------------------------------------
  legacy                           api
-------------------------------------------------------------
0: 593070M MV 5KKPALBG9DPFA6993 2013 WESTERN STAR 4900 SA  |
1: 156429L MV 5KKPALBG9DPFA6993 2013 WESTERN STAR 4900SA TRI  |
#######################################################################

### Similar Matches

#### Pass rate

In [ ]:
searches_total = 0
searches_passed = 0
failed_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        # fails if missed matches
        if len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > 0:
            failed_searches.append(search)
        # fails if order is off
        elif search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value) != 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

#### Failed searches

*select failed searches to analyze*

In [ ]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesSimilar']['resultsLegacy']), ' missed: ', len(json['matchesSimilar']['missedMatches']))
    print('first fail index: ', json['matchesSimilar']['firstFailIndex'])
    print('avg index diff: ', json['matchesSimilar']['avgIndexDiff'])

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [10]:
similar_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > _
        similar_searches_analysis.append(search)

*first failed indexes*

In [11]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

0


In [ ]:
### avgs
total_no_fails = 0
total_fail_index = 0
for search in similar_searches_analysis:
    fail_index = search.fail_index(TestSearchResult.MatchType.SIMILAR.value)
    if fail_index == -1:
        total_no_fails += 1
    else:
        total_fail_index += fail_index

num_searches_failed = len(similar_searches_analysis) - total_no_fails
print('Number of searches with fail indexes: ', num_searches_failed)
print('avg fail index: ', total_fail_index/num_searches_failed)

*avg order difference between legacy and api results (does NOT include missed matches)*

In [ ]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

*missed matches*

In [ ]:
import re
### manually iterate through
search = similar_searches_analysis[0]
print('criteria: ', search.search_criteria)
print('Missed Matches')
for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
    match['details'] = re.sub(' +', ' ', match['details'])
    print('-------------------------------------------------------------------------------')
    print('result:', match['details'])
    print('reg num:', match['documentId'])
    print('index: ', match['index'])

### print out all of them
# for search in similar_searches_analysis:
#     print('##################################################################################')
#     print('criteria: ', search.search_criteria)
#     print('Missed Matches')
#     for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
#         print('-------------------------------------------------------------------------------')
#         print('result:', match['details'])
#         print('reg num:', match['documentId'])
#         print('index: ', match['index'])

*results diff*

In [ ]:
import re
### manually iterate through similar_searches_analysis one by one
search = similar_searches_analysis[0]
print('criteria: ', search.search_criteria)
print('-------------------------------------------------------------')
print('  legacy                           api')
print('-------------------------------------------------------------')
legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
length = max(len(legacy_results), len(api_results))
for i in range(length):
    if i < len(legacy_results) and i < len(api_results):
        legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
        print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
    elif i < len(legacy_results):
        legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
        print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
    elif i < len(api_results):
        print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
# for search in similar_searches_analysis:
#     print('criteria: ', search.search_criteria)
#     print('-------------------------------------------------------------')
#     print('  legacy                           api')
#     print('-------------------------------------------------------------')
#     legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
#     api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
#     length = max(len(legacy_results), len(api_results))
#     for i in range(length):
#         if i < len(legacy_results) and i < len(api_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#         elif i < len(legacy_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#         elif i < len(api_results):
#             print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### Create MD file of notebook run
**NOTE:** save notebook (i.e. _cmd s_) now to have results show in markdown file

In [17]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [18]:
md_name = nb_name[:-6]+'.md'

In [ ]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2